In [1]:
import openpathsampling as paths
storage = paths.storage.Storage("mistis.nc", "r")

In [2]:
mistis = storage.networks.load(0)

In [3]:
for t in mistis.input_transitions:
    print repr(t)

In [4]:
print mistis.movers['repex']

[<openpathsampling.pathmover.ReplicaExchangeMover object at 0x110349c10>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x110302ed0>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x1105c2090>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x11031a490>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x110349310>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x110359c50>, <openpathsampling.pathmover.ReplicaExchangeMover object at 0x1105c2890>]


In [8]:
# this should be true if we restored correctly
for ens in mistis.sampling_transitions[0].ensembles:
    print ens in storage.ensembles[:]

True
True
True
